## Import Modules

In [412]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import random
from PIL import Image
from tqdm.notebook import tqdm
warnings.filterwarnings('ignore')
%matplotlib inline

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.image import load_img
from keras.src.legacy.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.optimizers import Adam

## Load the Dataset

In [413]:
DATASET = 'affectnet_balanced'
# DATASET = 'balanced_filtered_FER2013'
# DATASET = 'mma'
# DATASET = 'merged_dataset_balanced'

TRAIN_DIR = f"input/{DATASET}/train/"
TEST_DIR = f"input/{DATASET}/test/"
VALID_DIR = f"input/{DATASET}/valid/"

batch_size = 32
epochs = 100
learning_rate = 0.0001

In [414]:
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [415]:
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    color_mode='grayscale',
    target_size=(48, 48),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

valid_generator = valid_datagen.flow_from_directory(
    VALID_DIR,
    color_mode='grayscale',
    target_size=(48, 48),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    color_mode='grayscale',
    target_size=(48, 48),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

Found 18316 images belonging to 7 classes.
Found 5238 images belonging to 7 classes.
Found 2617 images belonging to 7 classes.


## Feature Extraction

In [416]:
labels = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

In [417]:
## convert label to integer
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(labels)

LabelEncoder()

In [418]:
input_shape = (48, 48, 1)
output_class = len(labels)

## Model Creation

In [419]:
# 5 layer model
model = Sequential(name='5-layer')

# convolutional layers
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', input_shape=input_shape, padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(1024, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
model.add(Dropout(0.25))

model.add(Conv2D(1024, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# flatten layer
model.add(Flatten())

# fully connected layers
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# output layer
model.add(Dense(output_class, activation='softmax'))

# _______________________________________________________

# # 6 layer model
# model = Sequential(name='6-layer')

# # Convolutional layers
# model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', input_shape=input_shape, padding='same'))
# model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
# model.add(Dropout(0.25))

# model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
# model.add(BatchNormalization())
# model.add(MaxPooling2D((2, 2), padding='same'))
# model.add(Dropout(0.25))

# model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'))
# model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
# model.add(Dropout(0.25))

# model.add(Conv2D(1024, kernel_size=(3, 3), activation='relu', padding='same'))
# model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
# model.add(Dropout(0.25))

# model.add(Conv2D(1024, kernel_size=(3, 3), activation='relu', padding='same'))
# model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
# model.add(Dropout(0.25))

# model.add(Conv2D(2048, kernel_size=(3, 3), activation='relu', padding='same'))
# model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(2, 2), padding='same'))
# model.add(Dropout(0.25))

# # global average pooling layer
# model.add(GlobalAveragePooling2D())

# # fully connected layers
# model.add(Dense(output_class, activation='softmax'))

# _______________________________________________________

# model = Sequential(name='YT_1')
# # soruce: https://www.youtube.com/watch?v=UHdrxHPRBng

# # convolutional layers
# model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))

# model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

# model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

# # flatten layer
# model.add(Flatten())

# # fully connected layer
# model.add(Dense(1024, activation='relu'))
# model.add(Dropout(0.5))

# # output layer
# model.add(Dense(output_class, activation='softmax'))

# _______________________________________________________

# model = Sequential(name='YT_4')
# # source: https://www.youtube.com/watch?v=Bb4Wvl57LIk

# # convolutional layers
# model.add(Conv2D(64,(3,3), padding = 'same', activation='relu', input_shape = input_shape))
# model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size = (2,2)))
# model.add(Dropout(0.25))

# model.add(Conv2D(128,(5,5),padding = 'same', activation='relu'))
# model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size = (2,2)))
# model.add(Dropout (0.25))

# model.add(Conv2D(512,(3,3),padding = 'same', activation='relu'))
# model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size = (2,2)))
# model.add(Dropout (0.25))

# model.add(Conv2D(512,(3,3), padding='same', activation='relu'))
# model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(2, 2)))
# model.add(Dropout(0.25))

# # flatten layer
# model.add(Flatten())

# # fully connected layers
# model.add(Dense(256, activation='relu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.25))

# model.add(Dense(512, activation='relu'))
# model.add(BatchNormalization())
# model.add(Dropout(0.25))

# # output layer
# model.add(Dense(output_class, activation='softmax'))

# _______________________________________________________

# model = Sequential(name='YT_5')
# # source: https://www.youtube.com/watch?v=mj-3vzJ4ZVw

# # convolutional layers
# model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=input_shape))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Dropout(0.4))

# model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Dropout(0.4))

# model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Dropout(0.4))

# model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Dropout(0.4))

# # flatten layer
# model.add(Flatten())

# # fully connected layers
# model.add(Dense(512, activation='relu'))
# model.add(Dropout(0.4))

# model.add(Dense(256, activation='relu'))
# model.add(Dropout(0.3))

# # output layer
# model.add(Dense(output_class, activation='softmax'))

In [420]:
checkpoint = ModelCheckpoint(
    f'models/checkpoints/{model.name}_{DATASET}.keras',
    monitor='val_acc',
    verbose=1,
    save_best_only=True,
    mode='max'
)

early_stopping = EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=5,
    verbose=1,
    restore_best_weights=True
)

reduce_learning_rate = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=2,
    verbose=1,
    min_delta=0.0001
)

callbacks = [reduce_learning_rate, early_stopping, checkpoint]

model.compile(
    optimizer=Adam(learning_rate = learning_rate),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

Model: "5-layer"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_149 (Conv2D)                  │ (None, 48, 48, 128)         │           1,280 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_147              │ (None, 48, 48, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_149 (MaxPooling2D)     │ (None, 24, 24, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_213 (Dropout)                │ (None, 24, 24, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_150 (Conv2D)                  │ (None, 24, 24, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_148              │ (None, 24, 24, 256)         │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_150 (MaxPooling2D)     │ (None, 12, 12, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_214 (Dropout)                │ (None, 12, 12, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_151 (Conv2D)                  │ (None, 12, 12, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_149              │ (None, 12, 12, 512)         │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_151 (MaxPooling2D)     │ (None, 6, 6, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_215 (Dropout)                │ (None, 6, 6, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_152 (Conv2D)                  │ (None, 6, 6, 1024)          │       4,719,616 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_150              │ (None, 6, 6, 1024)          │           4,096 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_152 (MaxPooling2D)     │ (None, 3, 3, 1024)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_216 (Dropout)                │ (None, 3, 3, 1024)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_153 (Conv2D)                  │ (None, 3, 3, 1024)          │       9,438,208 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_151              │ (None, 3, 3, 1024)          │           4,096 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_153 (MaxPooling2D)     │ (None, 1, 1, 1024)          │              

 Total params: 16,307,207 (62.21 MB)

 Trainable params: 16,299,783 (62.18 MB)

 Non-trainable params: 7,424 (29.00 KB)

## Train the model

In [ ]:
history = model.fit(
    train_generator,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=valid_generator,
    callbacks=callbacks
)

Epoch 1/100
573/573 ━━━━━━━━━━━━━━━━━━━━ 249s 428ms/step - accuracy: 0.1646 - loss: 2.9977 - val_accuracy: 0.1325 - val_loss: 2.6172 - learning_rate: 1.0000e-04
Epoch 2/100
573/573 ━━━━━━━━━━━━━━━━━━━━ 224s 391ms/step - accuracy: 0.2084 - loss: 2.5210 - val_accuracy: 0.3200 - val_loss: 1.6637 - learning_rate: 1.0000e-04
Epoch 3/100
573/573 ━━━━━━━━━━━━━━━━━━━━ 222s 387ms/step - accuracy: 0.2980 - loss: 2.0022 - val_accuracy: 0.3574 - val_loss: 1.4665 - learning_rate: 1.0000e-04
Epoch 4/100
573/573 ━━━━━━━━━━━━━━━━━━━━ 223s 389ms/step - accuracy: 0.3386 - loss: 1.7762 - val_accuracy: 0.3898 - val_loss: 1.4341 - learning_rate: 1.0000e-04
Epoch 5/100
573/573 ━━━━━━━━━━━━━━━━━━━━ 223s 390ms/step - accuracy: 0.3543 - loss: 1.6808 - val_accuracy: 0.4401 - val_loss: 1.3668 - learning_rate: 1.0000e-04
Epoch 6/100
573/573 ━━━━━━━━━━━━━━━━━━━━ 223s 389ms/step - accuracy: 0.3897 - loss: 1.5383 - val_accuracy: 0.4595 - val_loss: 1.3335 - learning_rate: 1.0000e-04
Epoch 7/100
573/573 ━━━━━━━━━━━━━━

## Save the model

In [ ]:
model.save(f'models/{model.name}_{DATASET}.keras')

## Plot the Results

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
epochs = range(len(acc))

plt.plot(epochs, acc, 'b', label='Training Accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation Accuracy')
plt.title('Accuracy Graph')
plt.legend()
plt.savefig(f'plots/AccuracyGraph_{model.name}_{DATASET}.png', bbox_inches='tight')
plt.figure()

loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(acc))

plt.plot(epochs, loss, 'b', label='Training Loss')
plt.plot(epochs, val_loss, 'r', label='Validation Loss')
plt.title('Loss Graph')
plt.legend()
plt.savefig(f'plots/LossGraph_{model.name}_{DATASET}.png', bbox_inches='tight')

## Accuracy, F1, Precision, Recall

In [ ]:
x_test = []
y_test = []

for i in range(len(test_generator)):
    x_batch, y_batch = test_generator[i]
    x_test.extend(x_batch)
    y_test.extend(np.argmax(y_batch, axis=1))

x_test = np.array(x_test)
y_test = np.array(y_test)

y_pred_prob = model.predict(x_test)

y_pred = np.argmax(y_pred_prob, axis=1)

In [ ]:
from sklearn.metrics import classification_report

report = classification_report(y_test, y_pred, target_names=labels)
print(report)

In [ ]:
from sklearn.metrics import accuracy_score

acc = accuracy_score(y_test, y_pred)
print(acc)

## Confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(
    le.inverse_transform(y_test),
    le.inverse_transform(y_pred),
    normalize='true'
)

sns.heatmap(
    cm,
    annot=True,
    xticklabels=labels,
    yticklabels=labels,
    cmap=sns.color_palette('Blues', 12),
    fmt='.2f'
)
plt.xlabel('Predicted label')
plt.ylabel('True label')

plt.title('Normalized confusion Matrix')
plt.savefig(f'plots/ConfusionMatrix_{model.name}_{DATASET}.png', bbox_inches='tight')
plt.figure()